In [2]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [3]:
pip install langchain langchain-community langchain-groq python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
import json
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

base_prompt = PromptTemplate.from_template(
    "Answer the following question concisely and accurately:\n\nQuestion: {question}"
)

web_prompt = PromptTemplate.from_template(
    "You are given the following web search results:\n\n{web_content}\n\nUse them to answer the following question:\n\nQuestion: {question}"
)

def web_search(query: str) -> str:
    url = "https://google.serper.dev/search"
    headers = {
        "X-API-KEY": os.environ.get("SERPER_API_KEY"),
        "Content-Type": "application/json"
    }
    payload = {
        "q": query,
        "gl": "in"
    }

    response = requests.post(url, headers=headers, json=payload)

    data_dict = response.json()
    organic_results = data_dict.get('organic', [])

    return "\n\n".join([
        f"{item.get('title', '')}\n{item.get('link', '')}"
        for item in organic_results[:3]
    ])

def needs_web_search(query: str) -> bool:
    trigger_keywords = ["weather", "news", "today", "latest", "now", "current",
        "temperature", "score", "headlines", "update", "live", "price", "stock"]
    return any(word in query.lower() for word in trigger_keywords)

def web_browsing_agent(question: str) -> str:
    if needs_web_search(question):
        print("\nPerforming web search...")
        web_content = web_search(question)
        final_prompt = web_prompt.format(web_content=web_content, question=question)
    else:
        print("\nUsing Groq LLM knowledge only...")
        final_prompt = base_prompt.format(question=question)

    return llm.invoke(final_prompt)


print("Web-Browsing Agent\n")
question = "What is the weather in Delhi today?"
answer = web_browsing_agent(question)
print("\nAnswer:\n", answer)

Web-Browsing Agent


Performing web search...

Answer:
 content='To find the current weather in Delhi, you can check the following websites:\n\n1. Accuweather: https://www.accuweather.com/en/in/delhi/202396/weather-forecast/202396\n2. Weather and Radar: https://www.weatherandradar.in/weather/delhi/18252928\n3. BBC Weather: https://www.bbc.com/weather/1261481\n\nPlease visit one of these websites for the most up-to-date and accurate information on the current weather in Delhi.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 133, 'total_tokens': 239, 'completion_time': 0.203619932, 'prompt_time': 0.013961432, 'queue_time': 0.051102017, 'total_time': 0.217581364}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--c8625872-dd03-4727-8181-aec9ace0a9da-0' usage_metadata={'input_tokens': 133, 'output_tokens': 106, 'total_tokens': 23

In [7]:
print("Web-Browsing Agent\n")
question = "Who won the world cup in 2022?"
answer = web_browsing_agent(question)
print("\nAnswer:\n", answer)

Web-Browsing Agent


Using Groq LLM knowledge only...

Answer:
 content='Argentina won the 2022 FIFA World Cup, defeating France 4-2 in a penalty shootout after the match ended 3-3 after extra time.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 57, 'total_tokens': 90, 'completion_time': 0.024480913, 'prompt_time': 0.021284554, 'queue_time': 0.047035286, 'total_time': 0.045765467}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--47fe624b-456b-4ac6-9fac-266063990433-0' usage_metadata={'input_tokens': 57, 'output_tokens': 33, 'total_tokens': 90}
